# setup

In [1]:
import pandas as pd
import numpy as np
import ast

# draft

In [2]:
path = 'ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'

In [3]:
Y = pd.read_csv(path + 'ptbxl_database.csv')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [4]:
AGG = pd.read_csv(path + 'scp_statements.csv', index_col = 0)
AGG = AGG[AGG.diagnostic == 1]

In [5]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in AGG.index:
            tmp.append(AGG.loc[key].diagnostic_class)
    return list(set(tmp))

In [6]:
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [7]:
columns = ['exam_id', 'NORM', 'MI', 'STTC', 'CD', 'HYP']

In [8]:
def label2onehot(exam_id, labels):
    onehot = [np.nan, False, False, False, False, False]
    onehot[0] = exam_id
    for label in labels:
        if label == 'NORM':
            onehot[1] = True
        if label == 'MI':
            onehot[2] = True
        if label == 'STTC':
            onehot[3] = True
        if label == 'CD':
            onehot[4] = True
        if label == 'HYP':
            onehot[5] = True
    return onehot

In [17]:
data = []
for idx, row in Y.iterrows():
    exam_id = row['ecg_id']
    labels = row['diagnostic_superclass']
    patient_id = int(row['patient_id'])
    fold = row['strat_fold']
    # data.append([exam_id, labels])
    # data.append(label2onehot(exam_id, labels))
    data.append(label2onehot(exam_id, labels) + [patient_id, fold])

In [18]:
data

[[1, True, False, False, False, False, 15709, 3],
 [2, True, False, False, False, False, 13243, 2],
 [3, True, False, False, False, False, 20372, 5],
 [4, True, False, False, False, False, 17014, 3],
 [5, True, False, False, False, False, 17448, 4],
 [6, True, False, False, False, False, 19005, 4],
 [7, True, False, False, False, False, 16193, 7],
 [8, False, True, False, False, False, 11275, 9],
 [9, True, False, False, False, False, 18792, 10],
 [10, True, False, False, False, False, 9456, 9],
 [11, True, False, False, False, False, 11243, 5],
 [12, True, False, False, False, False, 11031, 8],
 [13, True, False, False, False, False, 19953, 2],
 [14, True, False, False, False, False, 12925, 7],
 [15, True, False, False, False, False, 13375, 3],
 [16, True, False, False, False, False, 10999, 6],
 [17, False, False, False, False, False, 13619, 9],
 [18, False, False, False, False, False, 13619, 9],
 [19, True, False, False, False, False, 11116, 7],
 [20, False, False, False, False, Fals

In [21]:
metadata = pd.DataFrame(data, columns = columns + ['patient_id', 'fold'])
metadata

,exam_id,NORM,MI,STTC,CD,HYP,patient_id,fold
0,1,True,False,False,False,False,15709,3
1,2,True,False,False,False,False,13243,2
2,3,True,False,False,False,False,20372,5
3,4,True,False,False,False,False,17014,3
4,5,True,False,False,False,False,17448,4
...,...,...,...,...,...,...,...,...
21832,21833,False,False,True,False,False,17180,7
21833,21834,True,False,False,False,False,20703,4
21834,21835,False,False,True,False,False,19311,2
21835,21836,True,False,False,False,False,8873,8


In [22]:
metadata.to_csv('metadata.csv', index = None)